# Amazon Sagemaker SDK를 활용한 Sagemaker Pipelines 예제

<div class="alert alert-block alert-info">
⚠️ 이 노트북과 호환되는 것으로 알려진 최신 SageMaker Distribution 이미지 버전은 <code>3.1.0</code>입니다. 다른 버전에서 문제가 발생하면 버전 <code>3.1.0</code>으로 다운그레이드하세요. <b>이를 위해서는 JupyterApp을 중지하고, SageMaker Distribution 이미지를 <code>3.1.0</code>으로 다운그레이드한 다음 변경 사항을 적용하기 위해 JupyterLabApp을 재시작해야 합니다</b>.</div>

<div class="alert alert-warning"> 이 노트북은 SageMaker Managed MLflow가 실행 중인 환경을 필요로 합니다.</div>

이 예제는 데이터 전처리, 훈련, 평가, 레지스트리 단계가 포함된 SageMaker Pipeline을 구축하기 위해 Python 작업 공간을 모듈식으로 구성하는 방법을 보여줍니다.

### 1단계. SageMaker SDK의 최신 버전 및 로컬 요구 사항 설치

In [ ]:
!pip install -q --upgrade pip
!pip install -q -r local-requirements.txt

# make sure updates to the python modules are imported
%load_ext autoreload
%autoreload 2

## 설정 데이터 가져오기

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn

import boto3
import numpy as np
import pandas as pd
import os
import json

from IPython.display import Javascript, HTML

# Define session, role, and region so we can
# perform any SageMaker tasks we need
boto_session = boto3.Session()
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
sm_client = boto_session.client("sagemaker")

sagemaker.__version__

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')

if not space_name:
    raise Exception(f"Cannot find the current domain. Make sure you run this notebook in a JupyterLab in the SageMaker AI Studio")
else:
    print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

print(f"User profile name: {user_profile_name}")

### 2단계. config.yaml 업데이트
SageMaker Studio 외부에서 파이프라인을 실행하는 경우, 필요한 실행 역할을 제공해야 합니다.

In [ ]:
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']
sm_image = resource_spec.get('SageMakerImageArn', 'not defined')
sm_image_version = resource_spec.get('SageMakerImageVersionAlias', 'not defined')
print(f"""
SageMaker image: \033[1m{sm_image}\033[0m
SageMaker image version: \033[1m{sm_image_version}\033[0m
""")


실험에 관련된 정보를 기록하고 추적하기 위해 MLflow를 사용할 것입니다. MLflow는 중첩된 실행을 지원하므로 동일한 SageMaker Pipeline 실행으로 그룹화된 실행을 표시하는 데 유용합니다.

MLflow 추적 서버의 세부 정보를 찾아 변수에 저장해 보겠습니다.

In [ ]:
# Find an active MLflow server in the account
tracking_servers = [s['TrackingServerArn'] for s 
                    in sm_client.list_mlflow_tracking_servers()['TrackingServerSummaries']
                    if s['IsActive'] == 'Active']

if len(tracking_servers) < 1:
    print("You don't have any active MLflow servers. Trying to find a server in the status 'Creating'...")

    r = sm_client.list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Please create one")
        mlflow_server_arn = None
        mlflow_name = None
        raise('Missing MLflow server running')
    else:
        mlflow_server_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_server_arn = tracking_servers[0]
    mlflow_name = tracking_servers[0].split('/')[1]
    print(f"You have {len(tracking_servers)} running MLflow server(s). Get the first server ARN:{mlflow_server_arn}")

mlflow_experiment_name = "sm-pipeline-experiment"

## 구성 파일 생성

개별 단계를 실행하는 데 사용될 원격 함수 전반에 걸쳐 일관성을 보장하기 위해 `config.yaml`을 생성합니다. 예를 들어 전달할 수 있는 `ENV` 변수와 `requirements.txt`가 있습니다. 앞서 보았듯이, 사용할 컨테이너의 `image_uri`와 같은 더 많은 구성을 설정하는 것도 가능합니다.

In [ ]:
config_yaml = f"""
SchemaVersion: '1.0'
SageMaker:
  PythonSDK:
    Modules:
      TelemetryOptOut: true
      RemoteFunction:
        # role arn is not required if in SageMaker Notebook instance or SageMaker Studio
        # Uncomment the following line and replace with the right execution role if in a local IDE
        # RoleArn: <replace the role arn here>
        # image_uri: <replace with your image URI>
        InstanceType: ml.m5.xlarge
        EnvironmentVariables: {{'MLFLOW_TRACKING_URI': {mlflow_server_arn}, 'LOGNAME': {user_profile_name}, 'MLFLOW_EXPERIMENT_NAME': {mlflow_experiment_name}, 'AWS_REGION': {region}}}
        Dependencies: ./requirements.txt
        IncludeLocalWorkDir: True
        CustomFileFilter:
          IgnoreNamePatterns:
          - "data/*"
          - "models/*"
          - "*.ipynb"
          - "__pycache__"

"""

print(config_yaml, file=open('config.yaml', 'w'))
print(config_yaml)

## 3단계. 파일 분석

우리의 파이프라인은 4단계로 구성됩니다:
1. 데이터 전처리
2. 모델 훈련
3. 모델 출력 평가
4. 모델 등록

이러한 각 단계에 대해 전용 파일을 만들었습니다. 먼저 [`preprocess.py`](steps/preprocess.py)를 살펴보겠습니다.

In [ ]:
!pygmentize steps/preprocess.py

In [ ]:
!pygmentize steps/train.py

In [ ]:
!pygmentize steps/evaluation.py

In [ ]:
!pygmentize steps/register.py

## 로컬에서 단계 테스트

모든 함수가 작동하는지 확인하기 위해 로컬에서 테스트할 수 있습니다.

In [ ]:
import os
import mlflow

from steps.preprocess import preprocess
from steps.train import train
from steps.evaluation import evaluate
from steps.register import register

os.environ['MLFLOW_TRACKING_URI'] = mlflow_server_arn

bucket=sagemaker_session.default_bucket()
input_path = (f"s3://sagemaker-example-files-prod-{sagemaker_session.boto_region_name}/datasets"
              f"/tabular/uci_abalone/abalone.csv")

model_approval_status_param = "PendingManualApproval"

model_pkg_group_name = "abalone-model-new-sdk"

sm_pipeline_name = 'abalone-sm-pipeline-new-sdk'

mlflow.set_experiment(mlflow_experiment_name)

with mlflow.start_run(run_name=f"local-{sm_pipeline_name}") as run:
    run_id = run.info.run_id
    print(run_id)

In [ ]:
## preproces step
data = preprocess(input_path, run_id=run_id)

## training step
model = train(train_df=data[0], validation_df=data[1],run_id=run_id)

## evaluation step
evaluation_results = evaluate(model=model,test_df=data[2], run_id=run_id)

## model registration step
model_register=register(
    model=model,
    evaluation=evaluation_results,
    model_approval_status=model_approval_status_param,
    model_package_group_name=model_pkg_group_name,
    bucket=bucket,
    run_id=run_id
)

### 4단계. Amazon SageMaker Pipeline 실행

파이프라인을 로컬에서 성공적으로 실행한 후, SageMaker Pipelines에 의해 관리되는 서비스로 프로세스를 이동할 수 있습니다.

In [ ]:
!pygmentize pipeline.py

In [ ]:
!python pipeline.py --mlflow_tracking_uri {mlflow_server_arn} --mlflow_experiment_name {mlflow_experiment_name} --sagemaker_pipeline_name {sm_pipeline_name}

### 5단계. Amazon SageMaker Pipeline 실행 관찰

위 셀을 실행한 후 SageMaker Studio의 Pipeline 탭에서 SageMaker Pipeline의 실행을 볼 수 있습니다. 다음 그림에서 실행 예시를 확인할 수 있습니다:

![sm-pipeline-execution.png](attachment:25c2850e-d3e8-4fdd-924b-c7c2ae88cf06.png)

In [ ]:
from IPython.display import HTML

# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, sm_pipeline_name))
)

## 코딩 챌린지

현재 git 커밋 해시를 추적하도록 다음 코드를 수정하세요. **제안**:
1. GitPython 라이브러리를 사용하여 최신 커밋 해시를 찾습니다
2. [`pipeline.py`](pipeline.py) 파일을 수정하여 새 인수를 받고 해당 값을 mlflow에 기록합니다